### Peano numbers

In [1]:
abstract type Peano end

In [2]:
struct Zero <: Peano end

In [3]:
struct Succ{T<:Peano} <: Peano
	prev::T
end

In [4]:
typeof(Peano)

DataType

In [5]:
p0 = Zero()

Zero()

In [6]:
p1 = Succ(p0)

Succ{Zero}(Zero())

In [7]:
typeof(p1)

Succ{Zero}

In [8]:
p1.prev

Zero()

In [9]:
p2 = Succ(p1)

Succ{Succ{Zero}}(Succ{Zero}(Zero()))

In [10]:
p3 = Succ(p2)

Succ{Succ{Succ{Zero}}}(Succ{Succ{Zero}}(Succ{Zero}(Zero())))

In [11]:
Succ(Zero()) == p1

true

### `add`

In [12]:
add(x::Zero, y::Peano) = y

add (generic function with 1 method)

In [13]:
add(x::Succ, y::Peano) = Succ(add(x.prev, y))

add (generic function with 2 methods)

In [14]:
add(p2, p3)

Succ{Succ{Succ{Succ{Succ{Zero}}}}}(Succ{Succ{Succ{Succ{Zero}}}}(Succ{Succ{Succ{Zero}}}(Succ{Succ{Zero}}(Succ{Zero}(Zero())))))

In [15]:
@code_typed add(p1, p2)

CodeInfo(
1 ─     return $(QuoteNode(Succ{Succ{Succ{Zero}}}(Succ{Succ{Zero}}(Succ{Zero}(Zero())))))
) => Succ{Succ{Succ{Zero}}}

```text
ackermann(Z, n) = S(n);
ackermann(S(m), Z) = ackermann(m, S(Z));
ackermann(S(m), S(n)) = ackermann(m, a(S(m), n));
```

In [16]:
ackerman(m::Zero, n::Peano) = Succ(n)

ackerman (generic function with 1 method)

In [17]:
ackerman(m::Succ, n::Zero) = ackerman(m.prev, Succ(Zero()))

ackerman (generic function with 2 methods)

In [18]:
ackerman(m::Succ, n::Succ) = ackerman(m.prev, ackerman(m, n.prev))

ackerman (generic function with 3 methods)

In [19]:
ackerman(p3, p1)

Succ{Succ{Succ{Succ{Succ{Succ{Succ{Succ{Succ{Succ{Succ{Succ{Succ{Zero}}}}}}}}}}}}}(Succ{Succ{Succ{Succ{Succ{Succ{Succ{Succ{Succ{Succ{Succ{Succ{Zero}}}}}}}}}}}}(Succ{Succ{Succ{Succ{Succ{Succ{Succ{Succ{Succ{Succ{Succ{Zero}}}}}}}}}}}(Succ{Succ{Succ{Succ{Succ{Succ{Succ{Succ{Succ{Succ{Zero}}}}}}}}}}(Succ{Succ{Succ{Succ{Succ{Succ{Succ{Succ{Succ{Zero}}}}}}}}}(Succ{Succ{Succ{Succ{Succ{Succ{Succ{Succ{Zero}}}}}}}}(Succ{Succ{Succ{Succ{Succ{Succ{Succ{Zero}}}}}}}(Succ{Succ{Succ{Succ{Succ{Succ{Zero}}}}}}(Succ{Succ{Succ{Succ{Succ{Zero}}}}}(Succ{Succ{Succ{Succ{Zero}}}}(Succ{Succ{Succ{Zero}}}(Succ{Succ{Zero}}(Succ{Zero}(Zero())))))))))))))

In [20]:
(@which ackerman(p3, p1)).specializations

svec(MethodInstance for ackerman(::Succ{Succ{Succ{Zero}}}, ::Succ{Zero}), MethodInstance for ackerman(::Succ{Succ{Zero}}, ::Succ{Zero}), MethodInstance for ackerman(::Succ{Zero}, ::Succ{Zero}), MethodInstance for ackerman(::Succ{Zero}, ::Succ{Succ{Succ{Zero}}}), MethodInstance for ackerman(::Succ{Zero}, ::Succ{Succ{Zero}}), MethodInstance for ackerman(::Succ{Succ{Zero}}, ::Succ{Succ{Succ{Succ{Succ{Zero}}}}}), MethodInstance for ackerman(::Succ{Succ{Zero}}, ::Succ{Succ{Succ{Succ{Zero}}}}), MethodInstance for ackerman(::Succ{Succ{Zero}}, ::Succ{Succ{Succ{Zero}}}), MethodInstance for ackerman(::Succ{Succ{Zero}}, ::Succ{Succ{Zero}}), MethodInstance for ackerman(::Succ{Zero}, ::Succ{Succ{Succ{Succ{Succ{Zero}}}}}), MethodInstance for ackerman(::Succ{Zero}, ::Succ{Succ{Succ{Succ{Zero}}}}), MethodInstance for ackerman(::Succ{Zero}, ::Succ{Succ{Succ{Succ{Succ{Succ{Succ{Zero}}}}}}}), MethodInstance for ackerman(::Succ{Zero}, ::Succ{Succ{Succ{Succ{Succ{Succ{Zero}}}}}}), MethodInstance for ackerma

### `repeat`

In [21]:
repeat(n::Zero, a) = ()

repeat (generic function with 1 method)

In [22]:
repeat(n::Succ, a) = (a, repeat(n.prev, a))

repeat (generic function with 2 methods)

In [23]:
repeat(p3, 99)

(99, (99, (99, ())))

In [24]:
@code_typed repeat(p3, 99)

CodeInfo(
1 ─ %1 = Core.tuple(a, ())::Tuple{Int64, Tuple{}}
│   %2 = Core.tuple(a, %1)::Tuple{Int64, Tuple{Int64, Tuple{}}}
│   %3 = Core.tuple(a, %2)::Core.PartialStruct(Tuple{Int64, Tuple{Int64, Tuple{Int64, Tuple{}}}}, Any[Int64, Core.PartialStruct(Tuple{Int64, Tuple{Int64, Tuple{}}}, Any[Int64, Core.PartialStruct(Tuple{Int64, Tuple{}}, Any[Int64, Core.Const(())])])])
└──      return %3
) => Tuple{Int64, Tuple{Int64, Tuple{Int64, Tuple{}}}}

In [25]:
@code_native repeat(p3, 99)

	.text
; ┌ @ In[22]:1 within `repeat'
	movq	%rdi, %rax
	movq	%rsi, (%rdi)
	movq	%rsi, 8(%rdi)
	movq	%rsi, 16(%rdi)
	retq
	nop
; └


### `pw`

In [26]:
pw(n::Zero, x) = one(x)

pw (generic function with 1 method)

In [27]:
pw(n::Succ, x) = x * pw(n.prev, x)

pw (generic function with 2 methods)

In [28]:
@code_typed pw(p3, 99)

CodeInfo(
1 ─ %1 = Base.mul_int(x, 1)::Int64
│   %2 = Base.mul_int(x, %1)::Int64
│   %3 = Base.mul_int(x, %2)::Int64
└──      return %3
) => Int64

In [29]:
@code_native pw(p3, 99)

	.text
; ┌ @ In[27]:1 within `pw' @ In[27]:1
; │┌ @ int.jl:88 within `*'
	movq	%rdi, %rax
	imulq	%rdi, %rax
; │└
; │ @ In[27]:1 within `pw'
; │┌ @ int.jl:88 within `*'
	imulq	%rdi, %rax
; │└
	retq
	nopl	(%rax)
; └


### `Val(c)`

In [30]:
Val(99)

Val{99}()

In [31]:
typeof(Val(99))

Val{99}

In [32]:
typeof(Val((1,2,(3,4))))

Val{(1, 2, (3, 4))}

### Extracting `c` from `Val(c)`

In [33]:
get_val(::Val{n}) where {n} = n

get_val (generic function with 1 method)

In [34]:
get_val(Val(99))

99

In [35]:
get_val(Val((1,2,(3,4))))

(1, 2, (3, 4))

In [36]:
get_val(Val((:A, :B, :C)))

(:A, :B, :C)

### `pw(::Val{c}, x)`

In [37]:
pw_v(::Val{0}, x) = one(x)

pw_v (generic function with 1 method)

In [38]:
pw_v(::Val{n}, x) where {n} = x * pw_v(Val(n-1), x)

pw_v (generic function with 2 methods)

In [39]:
pw_v(n::Integer, x) = pw_v(Val(n), x)

pw_v (generic function with 3 methods)

In [40]:
pw_v(3, 2.0)

8.0

In [41]:
pw_v(5, 10)

100000

In [42]:
@code_typed pw_v(Val(5), 10)

CodeInfo(
1 ─ %1 = Base.mul_int(x, 1)::Int64
│   %2 = Base.mul_int(x, %1)::Int64
│   %3 = Base.mul_int(x, %2)::Int64
│   %4 = Base.mul_int(x, %3)::Int64
│   %5 = Base.mul_int(x, %4)::Int64
└──      return %5
) => Int64

### `one_of` - static `if`

Note that `k` is decreasing, in order to induce the compiler to unfold the calls to `one_of`.

In [43]:
function one_of(::Val{s}, ::Val{k}, x) where {s, k}
	if k == 0
		false
	elseif s[length(s) - k + 1] == x
		true
	else
		one_of(Val(s), Val(k-1), x)
	end
end

one_of (generic function with 1 method)

In [44]:
one_of(s::NTuple{N, T}, x) where {N, T} = one_of(Val(s), Val(N), x)

one_of (generic function with 2 methods)

In [45]:
one_of((:A, :B, :C), :B)

true

In [46]:
one_of((:A, :B, :C), :D)

false

In [47]:
@code_typed one_of(Val((:A, :B, :C)), Val(3), :D)

CodeInfo(
1 ──       goto #3 if not false
2 ──       nothing::Nothing
3 ┄─ %3  = (:A === x)::Bool
└───       goto #5 if not %3
4 ──       return true
5 ── %6  = (:B === x)::Bool
└───       goto #7 if not %6
6 ──       goto #11
7 ── %9  = (:C === x)::Bool
└───       goto #9 if not %9
8 ──       goto #10
9 ──       goto #10
10 ┄ %13 = φ (#8 => true, #9 => false)::Bool
└───       goto #11
11 ┄ %15 = φ (#6 => true, #10 => %13)::Bool
└───       return %15
) => Bool

In [48]:
@code_native one_of(Val((:A, :B, :C)), Val(3), :D)

	.text
; ┌ @ In[43]:1 within `one_of'
	movabsq	$139691867281648, %rax          # imm = 0x7F0C8C2098F0
; │ @ In[43]:4 within `one_of'
; │┌ @ Base.jl:87 within `=='
	cmpq	%rax, %rdi
	sete	%al
	movabsq	$139691867281688, %rcx          # imm = 0x7F0C8C209918
	cmpq	%rcx, %rdi
	sete	%cl
; │└
	orb	%al, %cl
	movabsq	$139691867759864, %rax          # imm = 0x7F0C8C27E4F8
	cmpq	%rax, %rdi
	sete	%al
	orb	%cl, %al
; │ @ In[43]:5 within `one_of'
	retq
	nopw	%cs:(%rax,%rax)
; └


### `StaticNumbers`

In [49]:
using StaticNumbers

In [50]:
d = 2

2

In [51]:
s = static(2)

static(2)

In [52]:
s + s

4

In [53]:
static(s + s)

static(4)

In [54]:
static(s+2)

static(4)

In [55]:
static(s + d)

static(4)

In [56]:
Tuple(i^2 for i in static(1):static(4)) # computed at compile time

(1, 4, 9, 16)

In [57]:
@code_typed Tuple(i^2 for i in static(1):static(4))

CodeInfo(
1 ─     invoke Base.power_by_squaring(static(1)::StaticInteger{1}, 2::Int64)::Int64
└──     return (1, 4, 9, 16)
) => NTuple{4, Int64}

In [58]:
@code_typed Tuple(i^2 for i in 1:4)

CodeInfo(
1 ─ %1 = invoke Base.collect(_2::Base.Generator{UnitRange{Int64}, var"#5#6"})::Vector{Int64}
│   %2 = Core._apply_iterate(Base.iterate, Core.tuple, %1)::Tuple{Vararg{Int64, N} where N}
└──      return %2
) => Tuple{Vararg{Int64, N} where N}

In [59]:
pw_s(n, x) = iszero(n) ? one(x) : x * pw_s(static(n-1), x)

pw_s (generic function with 1 method)

In [60]:
pw_s(3, "a")

"aaa"

In [61]:
@code_typed pw_s(3, 10)

CodeInfo(
1 ─ %1 = (n === 0)::Bool
└──      goto #3 if not %1
2 ─      return 1
3 ─ %4 = Base.sub_int(n, 1)::Int64
│   %5 = Core.apply_type(StaticNumbers.StaticInteger, %4)::Type{StaticInteger{_A}} where _A
│   %6 = (%5)()::StaticInteger{_A} where _A
│   %7 = Main.pw_s(%6, x)::Int64
│   %8 = Base.mul_int(x, %7)::Int64
└──      return %8
) => Int64

In [62]:
@code_typed pw_s(static(3), 10)

CodeInfo(
1 ─      goto #3 if not false
2 ─      nothing::Nothing
3 ┄ %3 = Base.mul_int(x, 1)::Int64
│   %4 = Base.mul_int(x, %3)::Int64
│   %5 = Base.mul_int(x, %4)::Int64
└──      return %5
) => Int64

### Ackermann

In [63]:
function A(m,n)
    if iszero(m)
        n + one(n)
    elseif iszero(n)
        A(@stat(m - one(m)), one(n))
    else
        A(@stat(m - one(m)), A(m, n - one(n)))
    end
end

A (generic function with 1 method)

In [64]:
using BenchmarkTools

In [65]:
@btime A(4, 1)

  13.930 s (0 allocations: 0 bytes)


65533

In [66]:
@btime A(static(4), 1)

  2.035 s (0 allocations: 0 bytes)


65533